In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import urllib
from tqdm.notebook import tqdm
import time
from datetime import datetime

# 1 Часть - основные города

In [2]:
def checkEnd(url):
    the_string = url.rsplit('php/',1)[1]
    srch = re.search('[а-яА-Я]', the_string) 
    if srch == None:
        str_url = urllib.parse.unquote(the_string)
    else:
        str_url = the_string
    #
    return str_url

In [3]:
def start(url):
    str_url = checkEnd(url)
    lst_splt = str_url.split("_")

    nmbr = lst_splt[-1]
    # 1 - тип транспорта
    tp_ts = lst_splt[-2]

    lst_type_ok = ['автобус','трамвай','троллейбус']

    i=0
    while not (any(map(str.isdigit, nmbr)) or ("б/н" in nmbr)):
        try:
            nmbr = lst_splt[i]
            if (any(map(str.isdigit, nmbr)) or ("б/н" in nmbr)):
                break
            i+=1
        except:
            nmbr = 0
            break
    #
    try:
        nmbr = nmbr.replace("-", "_")
    except:
        pass

    i=0
    while not tp_ts in lst_type_ok:
        try:
            tp_ts = lst_splt[i]
            if tp_ts in lst_type_ok:
                break
            i+=1
        except:
            tp_ts = "автобус другой"
            break

    # 
    return nmbr,tp_ts

In [4]:
def getTable(table_rows):
    l = []
    for tr in table_rows:
        try:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
        except:
            pass
    #
    return l

In [5]:
def getTable2(table_rows):
    def has_digit(strng):
        return bool(re.search(r'\d', strng))
    #
    lst_dgnt = ['_0','_1','_2','_3','_4','_5','_6','_7','_8','_9',
                '0_','1_','2_','3_','4_','5_','6_','7_','8_','9_']
    lst_dgt = ['0','1','2','3','4','5','6','7','8','9',
               '0','1','2','3','4','5','6','7','8','9']
    #
    l = []
    cnt=0
    for tr in table_rows:
        try:
            td = tr.find_all('td')
            row=[]
            for tr in td:
                if (("дни недел" not in tr.text) & ("примечан" not in tr.text) & (cnt==1)):
                    try:
                        clsp = int(tr.get("colspan"))
                    except:
                        clsp = 1
                        pass
                else:
                    clsp = 1
                #
                for j in range(clsp):
                    kk=0
                    for kk in range(len(tr.find_all('br'))):
                        try:
                            tr.br.replaceWith("_")
                        except:
                            pass
                    #
                    strng = tr.text
                    if has_digit(strng):
                        dg=0
                        for dg in range(len(lst_dgnt)):
                            strng = strng.replace(lst_dgnt[dg],lst_dgt[dg])
                    #
                    row.append(strng)
    #         row = [tr.text for tr in td]
            l.append(row)
        except:
            pass
        cnt+=1
        
    #
    return l

In [6]:
def find_time(str1, splttr):

    if splttr in str1:
        if splttr != "мин":
            tm_str = str1.split(splttr)[1].split("мин")[0].replace(" ","")
        else:
            tm_str = str1.split("мин")[0].replace(" ","")
        lst_dgt = re.findall(r'\d+', tm_str)
        try:
            time_min = int(lst_dgt[0])
        except:
            time_min = 0
        try:
            time_max = int(lst_dgt[1])
        except:
            time_max = time_min
        avg_time = int((time_min + time_max) / 2)
    else:
        avg_time = 0

    return avg_time
#

In [7]:
def get_time(interv):
    
    tm_avg_w = find_time(interv, "рабочи")
    if tm_avg_w == 0:
        tm_avg_w = find_time(interv, "мин")
    #
    tm_avg_h = find_time(interv, "выходны")
    tm_avg_evn = find_time(interv, "вечерн")
    if tm_avg_h == 0:
        tm_avg_sb = find_time(interv, "суббот")
        tm_avg_vs = find_time(interv, "воскресен")
        tm_avg_h = int((tm_avg_sb + tm_avg_vs) / 2)
    # 
    if tm_avg_evn == 0:
        tm_avg_evn=tm_avg_w
    if tm_avg_h == 0:
        tm_avg_h = tm_avg_w
    return tm_avg_w,tm_avg_h,tm_avg_evn

In [8]:
def get_sched(lst2):
    lst_ind = []
    if "час" in lst2[1][0]:
        if "минут" in lst2[2][0]:
            j=0
            for j in range(len(lst2[2])):
                if (lst2[2][j] != '\n') & (lst2[2][j] != ' \n'):
                    if ("час" not in lst2[2][j]) and ("мин" not in lst2[2][j]):
                        lst_ind.append(j)
    #
    lst_time=[]
    for k in lst_ind:
        hr = lst2[1][k].replace("\n","").replace(" ","")
        mnt = lst2[2][k].replace("\n","").replace(" ","")
        
        try:
            lst_dgt = re.findall(r'\d+', mnt)
            mnt = lst_dgt[0]
        except:
            pass
        if len(mnt) > 2:
            lst_mnt=[]
            chunks = int(len(mnt)/2)
            chunk_size = 2
            
            i=0
            for i in range(0, len(mnt), chunk_size):
                nwmnt = mnt[i:i+chunk_size]
                ttl_time = str(hr) + ":" + str(nwmnt)
                lst_time.append(ttl_time)
                
            #
        else:
            if mnt != "-":
                ttl_time = str(hr) + ":" + str(mnt)
                lst_time.append(ttl_time)
    # 
    lst_info_time =[]
    place_from = lst2[0][0].split("от ")[1].replace("\n","")
    lst_pf = []
    i=0
    for i in range(len(lst_time)):
        lst_pf.append(place_from)
    # 
    return lst_pf,lst_time

In [9]:
def end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,tmng,days,url):
    
    lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
    
    def foo(nlst,i):
        return nlst[i][1]

    for i in range(9):
        try: 
            tmp = foo(nlst,i)
        except:
            tmp = None
        lst_all.append(tmp)
    # 
    lst_all = lst_all + tmng
    lst_all = lst_all + days
    lst_all.append(url)
    return lst_all

In [10]:
def check_days(work_time):
    if ("рабоч" in work_time) & ("выходн" not in work_time):
        mn,tu,we,th,fr,st,su = 1,1,1,1,1,0,0
    else:
        mn,tu,we,th,fr,st,su = 1,1,1,1,1,1,1
    #
    return mn,tu,we,th,fr,st,su

In [11]:
def getInfo(url,region,city):
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    nmbr, tp_ts = start(url)
    
    tm_avg_w,tm_avg_h,tm_avg_evn = 0,0,0
    time_start = None
    plcfrm = None
    
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    #####

    all_tables = bs.find_all("table")

    #######
    l2,l3,l4,l5,l6,l7,l8 = [],[],[],[],[],[],[]
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if 'Содержание'in l[0][0]:
            l2 = l
        elif ('направление'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if ('прям' in l[0][0]) | ('прям' in tmp):
                if '№п' not in tmp:
                    l3 = l
                else:
                    l5 = l
            elif ('обратн' in l[0][0]) | ('обратн' in tmp):
                if '№п' in tmp:
                    l6 = l
            #
        elif ('обратное'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if '№п' in tmp:
                l6 = l
        elif 'работы'in l[0][0]:
            l7 = l
            interv = l7[1][1]
            ii = 0
            for ii, elem in enumerate(l7):
                if "нтервал" in l7[ii][0]:
                    interv = l7[ii][1]
                    break
                else:
                    interv = l7[1][1]
                ii+=1
            #
            if ("расписан" in interv) | (interv  == '-\n'):
                ind = i
                lst = getTable(all_tables[ind])
                while "отправлен" not in lst[0][0]:
                    try:
                        lst = getTable(all_tables[ind])
                        if "отправлен" in lst[0][0]:
                            break
                        else:
                            ind+=1
                    except:
                        ind-=1
                        break
                #
                lst = getTable(all_tables[ind])
                lst_bg_tm = []
                lst_bg_pf = []
                lst_tm_all = []
                while "отправлен" in lst[0][0]:
                    lst_pf,lst_time = get_sched(lst)
                    lst_bg_pf = lst_bg_pf + lst_pf
                    lst_bg_tm = lst_bg_tm + lst_time

                    try:
                        lst = getTable(all_tables[ind])
                        ind+=1
                    except:
                        break
                # 
                lst_tm_all.append(lst_bg_pf)
                lst_tm_all.append(lst_bg_tm)
            else:
                lst_tm_all = []
        else:
            l8 = l
    #  

    ####
    # 3 - наименование маршрута
    str1 = l2[0][0]
    str1 = str1.replace("\n\n","\n")    
    rt_nm = str1.split("\n")[1][2:].split("маршрут ")[1]
    nmbr, rt_nm = rt_nm.split('"', 1)
    rt_nm = rt_nm[:-1]
    nmbr = nmbr[:-1]
    nmbr = nmbr.replace("№ ","").replace("№","").replace("-", "_").replace("/", "_")

    ####
    # 4 -маршрут следования
    route_frw = l3[0][1].replace("\n","")
    try:
        route_bckw = l3[1][1].replace("\n","")
    except:
        route_bckw = 'кольцевой'
    ####

    ####
    # 5,6 - список оп
    def stops(l):
    #
        str_all=''
        for row in l[2:]:
            rw = []
            for i in row:
                i = i.replace("\n","")
                rw.append(i)
            str1 = rw[1]
            str_all = str_all + "_" + str1
        #
        str_all = str_all[1:]
        return str_all
    # 
    stops_frw = stops(l5)
    try:
        stops_bckw = stops(l6)
    except:
        stops_bckw = 'кольцевой'
    ####

    ####
    # 7) Режим работы
    # 8) Интервалы движения
    # 9) Выпуск на линию
    # 10) Время в пути
    # 11) Протяжённость маршрута
    # 12) Обслуживающие предприятия
    # 13) Подвижной состав
    # 14) Стоимость проезда
    # 15) Место отстоя 

    nlst = []
    for row in l7:
        tmplst = []
        for j in row:
            j = j.replace("\n","")
            tmplst.append(j)
        nlst.append(tmplst)
    # 
    
    work_time,interv,vpsk,rt_time,length,orgnztr,podvst,money,plc_rst = None,None,None,None,None,None,None,None,None
    
    
    i=0
    for i in range(len(nlst)):
        if (("ремя работы" in nlst[i][0]) | ("ежим работы" in nlst[i][0])):
            work_time = nlst[i][1]
        elif "нтервал" in nlst[i][0]:
            interv = nlst[i][1]
            if ("расписан" in interv) | (interv  == "-\n"):
                tm_avg_w,tm_avg_h,tm_avg_evn = None,None,None
            else:
                tm_avg_w,tm_avg_h,tm_avg_evn = get_time(interv)
        elif "ыпуск" in nlst[i][0]:
            vpsk = nlst[i][1]
        elif "ремя в пути" in nlst[i][0]:
            rt_time = nlst[i][1]
        elif ("ротяжённость" in nlst[i][0]) | ("ротяженность" in nlst[i][0]):
            length = nlst[i][1]
        elif "бслуживающие" in nlst[i][0]:
            orgnztr = nlst[i][1]
        elif "одвижной" in nlst[i][0]:
            podvst = nlst[i][1]
        elif "тоимость проезда" in nlst[i][0]:
            if money == None:
                money = nlst[i][1]
        elif ("есто" in nlst[i][0]) & ("отсто" in nlst[i][0]):
            plc_rst = nlst[i][1]
    # 
    if len(lst_tm_all) > 0:
        amnt_row = len(lst_tm_all[0])
        lst_plcfrm = lst_tm_all[0]
        lst_time_start = lst_tm_all[1]
    else:
        amnt_row = 1
        lst_plcfrm = [None]
        lst_time_start = [None]
    #
    
    tmp_lst = [work_time,interv,vpsk,rt_time,length,orgnztr,podvst,money,plc_rst]
    

    nlst2=[]
    for i in range(len(tmp_lst)):
        tl = [0,tmp_lst[i]]
        nlst2.append(tl)
    # 
    nlst = nlst2
    
    tmng = [plcfrm,time_start,tm_avg_w,tm_avg_h,tm_avg_evn]
    mn,tu,we,th,fr,st,su = check_days(work_time)
    days=[mn,tu,we,th,fr,st,su]
    
#     lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,tmng,url)
#     lst_all = lst_all + [tm_avg_w,tm_avg_h,tm_avg_evn,time_start]
    # 
    lst_all = []
    l=0
    for l in range(amnt_row):
        plcfrm = lst_plcfrm[l]
        time_start = lst_time_start[l]
        tmng = [plcfrm,time_start,tm_avg_w,tm_avg_h,tm_avg_evn]
        one_lst = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,tmng,days,url)
        lst_all.append(one_lst)
    # 
    
    return lst_all

In [12]:
def getInfo2(url,region,city):
    # Сызрань спешл
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    nmbr, tp_ts = start(url)
    
    all_tables = bs.find_all("table")
    
        # 3 - наименование маршрута
    rt_nm = str(bs.find_all('b')[2]).replace("<b>","").replace("</b>","")

    def getLines(lst_s,nmb):
        lst_ok = []

        for i in range(len(lst_s)):
            lst1 = lst_s[i][nmb].split("\n")
            for s in lst1:
                if s != '':
                    lst_ok.append(s)
        # 
        return lst_ok

    def stops(lst_s,nmb):
        lst_ok = getLines(lst_s,nmb)

        str1=''
        for row in lst_ok:
            if row != '\n':
                row = row.replace("\n", "")
                str1 = str1 + "_" + row
        # 
        str1 = str1[1:]
        return str1

    ####
    
    i=0
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if "в сторону" in l[0][0]:
            l5 = getTable(all_tables[i+2])
            l6 = getTable(all_tables[i+3])
            break
        else:
            l5 = getTable(all_tables[5])
            l6 = getTable(all_tables[6])
    # 

    ####
    # 4 -маршрут следования

    route_frw = stops(l5,0).replace("\n","")
    try:
        route_bckw = stops(l6,0).replace("\n","")
    except:
        route_bckw = 'кольцевой'
    ####

    # 5,6 - список оп
    stops_frw = stops(l5,1)
    try:
        stops_bckw = stops(l6,1)
    except:
        stops_bckw = 'кольцевой'
    ####
    
    # nlst = []

    lst_fnd = bs.find_all(["ul","p"])
    orgnztr,podv_sost,plc_rst,money = '','','',''
    i=0
    for i in range(len(lst_fnd)):
        if "бслуживающ" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                orgnztr = str(orgnztr) + "_" + lst_fnd[i].text
                i+=1
        elif "одвижной состав" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                podv_sost = str(podv_sost) + "_" + lst_fnd[i].text
                i+=1
        elif "тоимость проезд" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                money = str(money) + "_" + lst_fnd[i].text
                i+=1
        elif "Обед" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                plc_rst = str(plc_rst) + "_" + lst_fnd[i].text
                i+=1
    # 
    orgnztr = orgnztr.replace("\n","").replace("None_","").replace("_","",1)
    podv_sost = podv_sost.replace("\n","").replace("None_","").replace("_","",1)
    plc_rst = plc_rst.replace("\n","").replace("None_","").replace("_","",1)
    money = money.replace("\n","").replace("None_","").replace("_","",1)
    
    tm_avg_w,tm_avg_h,tm_avg_evn = None,None,None
    time_start = None
    plcfrm = None
    tmng = [plcfrm,time_start,tm_avg_w,tm_avg_h,tm_avg_evn]
    mn,tu,we,th,fr,st,su = None,None,None,None,None,None,None
    days=[mn,tu,we,th,fr,st,su]

    nlst = [[0,None],[0,None],[0,None],[0,None],[0,None],[0,orgnztr],[0,podv_sost],[0,money],[0,plc_rst]]


    lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,tmng,days,url)
    lst_all = [lst_all]
    return lst_all

In [13]:
def checkGrpNm(tmp_grp_nm):
    
    if (tmp_grp_nm == 'маршруты') | ("городск" in tmp_grp_nm):
        grp_name = 'муниципальный'
    elif 'коммерческ' in tmp_grp_nm:
        grp_name = 'коммерческий'
    elif  'пригородн' in tmp_grp_nm:
        grp_name = 'пригородный'
    elif ("междугород" in tmp_grp_nm) | ('межмуницип' in tmp_grp_nm):
        grp_name = 'межмуниципальный'
    elif "сезон" in tmp_grp_nm:
        grp_name = 'сезонный'
    elif "специальн" in tmp_grp_nm:
        grp_name = 'специальный'
    else:
        grp_name = tmp_grp_nm
    # 
    return grp_name

In [14]:
# 
def linksCity(city):

    url_grp = 'http://www.samaratrans.info/wiki/index.php/' + city
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('a')
    links=[]
    for link in stfa:
        try:
            if 'маршрут' in link.get("title"):
                lsh = 'http://www.samaratrans.info' + link.get("href")
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [15]:
def routeLinks(url_grp):
    links=[]
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('table')
    
    cnt=2
    for i in range(len(stfa)):
        table_rows = stfa[i]
        l = getTable(table_rows)
        if 'Наименование маршрута\n' in l[0]:
            cnt = i
            break
    # 
    lst_find = stfa[cnt].find_all('a')
    for link in lst_find:
        try:
            lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
            lnk_hr = link.get("href")
            ch = checkEnd(lnk_hr)
            if not any((c in ch) for c in lst_notok):
                lsh = 'http://www.samaratrans.info' + lnk_hr
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [16]:
def getCityData(city):
    lst_links = []
    lst_links = linksCity(city)
    all_routes=[]
    for grp in range(len(lst_links)):
        tmp_grp_nm = checkEnd(lst_links[grp]).split("_",2)[-1]
        grp_name = checkGrpNm(tmp_grp_nm)
        
        try:
            lst_group = []
            lst_group = routeLinks(lst_links[grp])
        except:
            lst_group = []
        for rtl in range(len(lst_group)):
            try:
                tmp_lnk_nm = checkEnd(lst_group[rtl])
                lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
                if not any((c in tmp_lnk_nm) for c in lst_notok):
                    try:
                        one_route = getInfo(lst_group[rtl],grp_name,city)
                    except:
                        one_route = getInfo2(lst_group[rtl],grp_name,city)
                    ln=0
                    for ln in range(len(one_route)):
                        on_rt = one_route[ln]
                        all_routes.append(on_rt)
                #
            except:
                one_route = [lst_group[rtl]] * 31
                if one_route not in all_routes:
                    all_routes.append(one_route)
                #
    return all_routes
# 

In [17]:
def mainCities():
    lst_ct = ['Самара', 'Тольятти', 'Сызрань', 'Новокуйбышевск', 
          'Чапаевск', 'Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']
#     lst_ct = ['Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']
    lst_big = []
    for i in tqdm(range(len(lst_ct))):
        city = lst_ct[i]
        lst = getCityData(city)
        lst_big = lst_big + lst
    # 
    return lst_big

# 2 Часть - другие города (ссылка ниже)

In [18]:
def getOtherLinks(url_ot):
    req = requests.get(url_ot)
    bs = BeautifulSoup(req.text, 'html.parser')

    all_lnks = bs.find_all('a')
    lst_rasp=[]
    lst_txt=[]
    
    i=0
    for link in all_lnks:
        if "расписани" in str(link.get('title')):
            lsh = "http://www.samaratrans.info" + link.get('href')
            reg_name = link.text
            lst_txt.append(reg_name)
            lst_rasp.append(lsh)
    # 

    return lst_rasp,lst_txt

In [19]:
def time_table(lst_row,ind_clm,amnt_clm_ct):
#     amnt_clm_ct+=1
    
    lst_tm = []
    
    tm = lst_row[ind_clm]
    if (('-' not in tm) & ("н/д" not in tm)):
        if "(" in tm:
            tm = tm.split("(")[0]
        tm = tm.replace("\n","").replace(" ","").replace("*","")
        lst_tm.append(tm)
    else:
        return [],[]
    #
    lst_days = []
    if ("ежедневно\n" in lst_row) | (" ежедневно\n" in lst_row):
        lst_days = lst_days + [1,1,1,1,1,1,1]
    else:
        lst_chk_days = lst_row[amnt_clm_ct:]
        
        i=0
        for i in range(len(lst_chk_days)):
            day = lst_chk_days[i]
            if ((day == '\n') | (day == ' \n') | (day == '-\n')):
                lst_days.append(None)
            else:
                
                day1 = day.replace("\n","").replace(" ","")
                if "н/д" in day1:
                    day1 = 1
                try:
                    lst_dgt = re.findall(r'\d+', day1)
                    day1 = int(lst_dgt[0])
                    if day1 > 0:
                        day1 = 1
                    else:
                        day1 = 0
                except:
                    pass
                lst_days.append(day1)
                
        # 
        if len(lst_days) < 7:
            while len(lst_days) != 7:
                lst_days.append(None)
                if len(lst_days) == 7:
                    break
        # 
        if len(lst_days) > 7:
            while len(lst_days) != 7:
                lst_days = lst_days[:-1]
                if len(lst_days) == 7:
                    break
        # 
    #
    return lst_tm, lst_days

In [20]:
def one_rt_shed(lst1):

    i=0
    for i in range(len(lst1[0])):
        if 'скобк' in lst1[0][i]:
            ind_sk = 1
            break
        else:
            ind_sk = 0
    #
    
    i=0
    for i in range(len(lst1[0])):
        if 'дни недели' in lst1[0][i]:
            ind_dn = ind_dn + ind_sk
            break
        else:
            ind_dn = -2
    #
    i=0
    for i in range(len(lst1)):
        if (("пн\n" in lst1[i]) | ("вт\n" in lst1[i])):
            ind_tm = i+1
            break
        else:
            ind_tm = 2
    #
    lst_days_names = ['пн', 'вт', 'ср', 'чт', 'пт', 'сб','вс']
    lst_plc_tmp = lst1[ind_sk][:ind_dn]
    lst_plc = []
    i=0
    for i in range(len(lst_plc_tmp)):
        plc = lst_plc_tmp[i]
        try:
            plc = plc.replace("\n","")
        except:
            pass
        if plc not in lst_days_names:
            lst_plc.append(plc)
    #
    amnt_clm_ct = len(lst_plc)
    amnt_rws = len(lst1[ind_tm:])
    
    def check_amnt_ct(amnt_rws,amnt_clm_ct,lst1,ind_tm):
        i=0
        for i in range(amnt_rws):
            lr = lst1[ind_tm+i]
            if ":" in lr[amnt_clm_ct]:
                amnt_clm_ct+=1
                lst_plc.append("city")
                break
        #
        return amnt_clm_ct,lst_plc
    #
    cnt=0
    while cnt < len(lst1[ind_tm][amnt_clm_ct:]):
        amnt_clm_ct,lst_plc = check_amnt_ct(amnt_rws,amnt_clm_ct,lst1,ind_tm)
        cnt+=1
    # 
    
    # ind_clm = 1
    lst_tm, lst_days = [],[]
    bg_lst_tm,bg_lst_days,bg_lst_ct = [],[],[]
    i=0
    for i in range(amnt_rws):
        lst_row = lst1[ind_tm+i]
        if "отменён\n" not in lst_row:
            j=0
            for j in range(amnt_clm_ct):
                ind_clm = j
                lst_tm, lst_days = time_table(lst_row,ind_clm,amnt_clm_ct)
                if (len(lst_tm) > 0) & (len(lst_days) > 0):
                    bg_lst_tm = bg_lst_tm + lst_tm
    #                 bg_lst_days = bg_lst_days + lst_days
                    bg_lst_ct = bg_lst_ct + [lst_plc[j]]

    #                 bg_lst_tm.append(lst_tm)
                    bg_lst_days.append(lst_days)
    #                 bg_lst_ct.append([lst_plc[j]])
        # 
    return bg_lst_ct,bg_lst_tm,bg_lst_days

In [21]:
def getInfo3(url,city):

    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')

#     city = checkEnd(url).split("_",1)[0]

    ####
    # get stops
    # stops_frw

    all_p = bs.find_all('p')
    p=0
    lst_txt = []
    lst_rt_time=[]
    for p in range(len(all_p)):
        if "Промежуточные" in str(all_p[p]):
            lst_txt.append(all_p[p].text)
            try:
                if ("остановки: " in all_p[p].text) | ("остановки: " in all_p[p+1].text):
                    if 'Время в пути' in str(all_p[p+1]):
                        rt_time = all_p[p+1].text.replace("Время в пути:","").replace("\n","")
                        lst_rt_time.append(rt_time)
                    else:
                        rt_time = None
                        lst_rt_time.append(rt_time)
                else:
                    if 'Время в пути' in str(all_p[p+1]):
                        rt_time = all_p[p+1].text.replace("Время в пути:","").replace("\n","")
                        lst_rt_time.append(rt_time)
            except:
                pass
    #  
    ####
    # костыль!
    if url == "http://www.samaratrans.info/wiki/index.php/%D0%A7%D0%B5%D0%BB%D0%BD%D0%BE-%D0%92%D0%B5%D1%80%D1%88%D0%B8%D0%BD%D1%8B_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F":
        str_bad = "Промежуточные остановки: нет\n"
        lst_txt.remove(str_bad)
    # 

    ####
    
    i=0
    lst_stops=[]
    for i in range(len(lst_txt)):
        txt = lst_txt[i].split(":",1)[1].replace(", ","_").replace("\n","")
        if "остановки:" in lst_txt[i]:
            lst_stops.append(txt)
        else:
            txt2 = ''
            while "остановки:" not in lst_txt[i]:
                ii = i
                txt = lst_txt[i].replace(", ","_").replace("\n","").replace("Промежуточные остановки ","")
                txt2 = txt2 + "[!!!]" + txt
                i+=1
                if "остановки:" in lst_txt[i]:
                    if len(lst_stops) == 0:
                        lst_stops.append(txt2)
                        break
                    elif "[!!!]" not in lst_stops[-1]:
                        lst_stops.append(txt2)
                        break
    # 
    ####
    ####
    i=0
    all_tables = bs.find_all('table')
    for i in range(len(all_tables)):
        l = getTable(bs.find_all('table')[0])
        if "Расписание" in l[0][0]:
            l1 = l
            ind_tb = i+1
            break
        else:
            l1 = getTable(bs.find_all('table')[0])
            ind_tb=1
    # 
    lst_nms = l1[0][0].split("\n")
    ####
    
    lst_all_ct,lst_all_tm,lst_all_days = [],[],[]

    ptbl = bs.find_all(['p','table'])
    i=0
    for i in range(len(ptbl)):
        if ptbl[i].name == 'p':
            if "ромежуточные" in ptbl[i].text:
                while ptbl[i].name != 'table':
                    i+=1
                    if ptbl[i].name == 'table':
                        break
                #
                bg_lst_ct,bg_lst_tm,bg_lst_days = [],[],[]
                while ptbl[i].name == 'table':
                    lst1 = getTable2(ptbl[i])
                    if "часы" in lst1[1][0]:
                        lst_ct,lst_tm = get_sched(lst1)
                        lst_days = [[1,1,1,1,1,1,1]]*len(lst_ct)
                        bg_lst_ct = bg_lst_ct + lst_ct
                        bg_lst_tm = bg_lst_tm + lst_tm
                        bg_lst_days = bg_lst_days + lst_days

                    else:
                        bg_lst_ct,bg_lst_tm,bg_lst_days = one_rt_shed(lst1)
                    #
                    i+=1
                    if ptbl[i].name != 'table':
                        if ((("ромежуточные" in ptbl[i].text)
                             & ("ромежуточные" not in ptbl[i+1].text))
                            | ("атегории" in ptbl[i].text)
                            | (ptbl[i-1].name == 'table')):
                            if ((lst_all_ct == []) | (bg_lst_ct not in lst_all_ct[-1:])):
                                lst_all_ct.append(bg_lst_ct)
                                lst_all_tm.append(bg_lst_tm)
                                lst_all_days.append(bg_lst_days)
                            break
                        else:
                            i+=1
                #
    #


    ####
    lst_reg = []
    lst_tpts = []
    lst_rtnbr = []
    lst_rtnm = []
    for i in range(len(lst_nms)):
        str1 = lst_nms[i]
        if "Расписание" in str1:
            if (" муниципальн" in str1) | (" городск" in str1):
                region = "муниципальный"
            elif (" межмуниципальн" in str1) | (" межрегион" in str1) | (" пригород" in str1):
                region = "межмуниципальный"
            else:
                region = "другой"
        else:
            if "маршрут" in str1:
    #             lst_rt.append(str1)
                tp_ts, rt = str1.split(" ",1)[1].split("маршрут ",1)
                rt_nbr, rt_nm = rt.split('"',1)
                rt_nbr = rt_nbr.replace(" ", "").replace("№", "").replace("-", "_").replace("/", "_")
                rt_nm = rt_nm[:-1]

                if "втобус" in tp_ts:
                    tp_ts = "автобус"
                elif "рамвай" in tp_ts:
                    tp_ts = "трамвай"
                elif "роллейбус" in tp_ts:
                    tp_ts = "троллейбус"
                else:
                    tp_ts = "другой"
                # 
                lst_rtnbr.append(rt_nbr)
                lst_rtnm.append(rt_nm)
                lst_tpts.append(tp_ts)

                lst_reg.append(region)
    # 
    ####

    ####
    
    
    
    lst_all = []
    def foo(lst,nmbr):
        try:
            answ = lst[nmbr]
        except:
            answ = None
        return answ
    i=0
    for i in range(len(lst_stops)):
        tp_ts = lst_tpts[i]
        region = lst_reg[i]
        nmbr = lst_rtnbr[i]
        rt_nm = lst_rtnm[i]
        route_frw = None
        route_bckw = None
        stops_frw = lst_stops[i]
        stops_bckw = None
        nlst = [[0,None],[0,None],[0,None],[0,lst_rt_time[i]],[0,None],[0,None],[0,None],[0,None],[0,None]]
        
        tm_avg_w,tm_avg_h,tm_avg_evn = None,None,None
        
        try:
            l_time_start = lst_all_tm[i]
            l_plcfrm = lst_all_ct[i]
            l_days = lst_all_days[i]
        except:
            l_time_start = [None]
            l_plcfrm = [None]
            l_days = [[None,None,None,None,None,None,None]]
        #
        j=0
        for j in range(len(l_time_start)):
            plcfrm = l_plcfrm[j]
            time_start = l_time_start[j]
            days = l_days[j]
            tmng = [plcfrm,time_start,tm_avg_w,tm_avg_h,tm_avg_evn]
            small_lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,tmng,days,url)
            lst_all.append(small_lst_all)
    # 
    ####
    

    return lst_all

In [22]:
def smallCities():

    url_ot = "http://www.samaratrans.info/wiki/index.php/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F"

    lst_rasp,lst_txt = getOtherLinks(url_ot)

    lst_big2 = []
    i=0
    for i in tqdm(range(len(lst_rasp))):
        url = lst_rasp[i]
        city = lst_txt[i]
        try:
            lst = getInfo3(url,city)
        except:
            lst = [[url] * 31]
        #
        for j in range(len(lst)):
            lst_big2.append(lst[j])
    # 

    return lst_big2

In [23]:
def createReestr():

    lst_big = mainCities()
    lst_big2 = smallCities()

    clmn = ["населенный_пункт","тип_тс","тип_маршрута","номер_маршрута",
            "наим_маршрута","маршут_след_прям","маршут_след_обрат",
            "список_оп_прям","список_оп_обрат","режим_работы","интервалы","выпуск","время_в_пути",
            "протяженность","предприятие","подв_состав","стоимость_проезда","место_отстоя",
            "место_отправл","время_отправл","интерв_ср_буд","интерв_ср_вых", "интерв_ср_вчр",
             "пнд","втр","срд","чтв","птн","сбт","вск",
            "ссылка"]
    
    df = pd.DataFrame(data=lst_big,columns=clmn)
    df2 = pd.DataFrame(data=lst_big2,columns=clmn)

    reestr = df.append(df2).reset_index(drop=True)
    #
    lst_clm = ["интерв_ср_буд","интерв_ср_вых", "интерв_ср_вчр",
                 "пнд","втр","срд","чтв","птн","сбт","вск"]
    for clm in lst_clm:
        reestr[clm] = reestr[clm].fillna(0)
        try:
            reestr[clm] = reestr[clm].astype(int)
        except:
            print(clm)
    # 

    return reestr

In [29]:
%time reestr = createReestr()


Wall time: 4min 48s


In [30]:
# # можно посмотреть, сколько ошибок (если ошибка - в таблице ссылка на маршрут)
# # должно быть 0
lst_notok = list(set(list(reestr[reestr["населенный_пункт"].str.contains("www")]["ссылка"])))
print(len(lst_notok))
if len(lst_notok) == 0:
    str_date = "{:%d_%H%M}".format(datetime.now())
    reestr.to_csv("reestr_{}.csv".format(str_date),encoding='utf-8-sig',sep=';',index=False)
#

0


In [26]:
# city="Samara"
# region="Munic"
# url="http://www.samaratrans.info/wiki/index.php/%D0%98%D1%81%D0%B0%D0%BA%D0%BB%D1%8B_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F"


In [27]:
# req = requests.get(url)
# bs = BeautifulSoup(req.text,'html.parser')
# all_tables = bs.find_all('table')
# lst1 = getTable(all_tables[-1])